In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math
import functools
import collections
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import anderson
from termcolor import colored

from numpy.random import seed
from numpy.random import randn
from statsmodels.graphics.gofplots import qqplot

#plot
import matplotlib.pyplot as plt
import seaborn as sns

## Download data

In [17]:
path_ = r'D:\vm_exchange\PhD_Data\HealthAssessment'
df = pd.read_csv(os.path.join(path_,'HA_all_20-10-2020V3.csv'), sep=';', index_col=0) 
li_cont = ['CombPeck','CombColor','FeatherNeck','FeatherWing','FeatherTail','FeatherCloaca','CloacaWounds',
            'RFPododermatitis','RFinjuries','LFPododermatitis','LFinjuries']
li_cat = ['PassageID', 'BirdID', 'Diarrhea', 'MissingToes','RFBumblefoot','LFBumblefoot']
for c in li_cont:
    df[c] = df[c].map(lambda x: float(str(x).replace(',','.')))
print(df.shape)
df.head(3)

(72, 17)


,PassageID,BirdID,CombPeck,CombColor,FeatherNeck,FeatherWing,FeatherTail,FeatherCloaca,CloacaWounds,Diarrhea,MissingToes,RFPododermatitis,RFBumblefoot,RFinjuries,LFPododermatitis,LFBumblefoot,LFinjuries
Person,,,,,,,,,,,,,,,,,
Mark,1,16 gris,200.0,0.0,125.0,0.0,25.0,0.0,0.0,0,0,50.0,0,0.0,25.0,0,0.0
Camille,1,16 gris,200.0,0.0,75.0,0.0,0.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0.0
Mark,2,16 gris,200.0,0.0,25.0,0.0,0.0,0.0,0.0,0,0,50.0,0,0.0,50.0,0,0.0


## Visually

In [18]:
for v in li_cont:
    print('----------------- INITIAL ',v)
    plt.hist(df[v].dropna(), bins='auto')
    plt.title(v, size=8)    
    plt.savefig(os.path.join(path_,'Histogram_'+v+'.png'),dpi=300,format='png',bbox_inches='tight')  
    plt.close()
    #qqplot
    qqplot(df[v].dropna(), line='s')
    plt.title(v, size=8)
    plt.savefig(os.path.join(path_,'qqplot-'+v+'.png'),dpi=300,format='png',bbox_inches='tight') 
    plt.close()
    
'''    print('----------------- LOG ',v)
    data = np.log(list(df[v].dropna()))
    if 0 in list(df[v].dropna()):
        data = np.log(list(df[v].dropna()+1))
    plt.hist(data, bins='auto')
    plt.title(v, size=8)
    plt.savefig(os.path.join(path_,'Histogram_LOGTransformation_'+v+'.png'),dpi=300,format='png',bbox_inches='tight')  
    plt.close()
    #qqplot
    qqplot(np.log(list(df[v].dropna())), line='s')
    plt.title(v, size=8)
    plt.savefig(os.path.join(path_,'qqplot-LOGTransformation_'+v+'.png'),dpi=300,format='png',bbox_inches='tight')
    plt.close()'''

----------------- INITIAL  CombPeck
----------------- INITIAL  CombColor
----------------- INITIAL  FeatherNeck
----------------- INITIAL  FeatherWing
----------------- INITIAL  FeatherTail
----------------- INITIAL  FeatherCloaca
----------------- INITIAL  CloacaWounds
----------------- INITIAL  RFPododermatitis
----------------- INITIAL  RFinjuries
----------------- INITIAL  LFPododermatitis
----------------- INITIAL  LFinjuries


"    print('----------------- LOG ',v)\n    data = np.log(list(df[v].dropna()))\n    if 0 in list(df[v].dropna()):\n        data = np.log(list(df[v].dropna()+1))\n    plt.hist(data, bins='auto')\n    plt.title(v, size=8)\n    plt.savefig(os.path.join(path_,'Histogram_LOGTransformation_'+v+'.png'),dpi=300,format='png',bbox_inches='tight')  \n    plt.close()\n    #qqplot\n    qqplot(np.log(list(df[v].dropna())), line='s')\n    plt.title(v, size=8)\n    plt.savefig(os.path.join(path_,'qqplot-LOGTransformation_'+v+'.png'),dpi=300,format='png',bbox_inches='tight')\n    plt.close()"

## Statistically

In [19]:
def ShapiroTest(data, alpha=0.05):
    '''The tests assume that that the sample was drawn from a Gaussian distribution. 
    p <= alpha: reject H0, not normal.
    p > alpha: fail to reject H0, normal'''
    # normality test
    stat, p = shapiro(data)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    if p > alpha:
        #print_color((('%.3f: %.3f' % (sl, cv),'black'),('data looks normal (fail to reject H0)','green')))
        print(colored('Sample looks Gaussian (fail to reject H0)','green'))
    else:
        print(colored('Sample does not look Gaussian (reject H0)','red'))
    return(stat, p)
    
def AgostinosK2Test(data, alpha=0.05):
    '''D’Agostino’s K^2 Test'''
    # normality test
    stat, p = normaltest(data)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    if p > alpha:
        print(colored('Sample looks Gaussian (fail to reject H0)','green'))
    else:
        print(colored('Sample does not look Gaussian (reject H0)','red'))
    return(stat, p)

def andersonTest(data):
    '''Anderson-Darling Test for normality test'''
    result = anderson(data)
    print('Statistic: %.3f' % result.statistic)
    p = 0
    for i in range(len(result.critical_values)):
        sl, cv = result.significance_level[i], result.critical_values[i]
        if result.statistic < result.critical_values[i]:
            print(colored('%.3f: %.3f' % (sl, cv),'grey'), colored('data looks normal (fail to reject H0)','green'))
        else:
            print(colored('%.3f: %.3f' % (sl, cv),'grey'), colored('data does not look normal (reject H0)','red'))
    return result  
        
        
for v in li_cont:
    print('------------------------ '+v)
    print('ShapiroTest')
    stat, p = ShapiroTest(list(df[v].dropna()))
    print('D’Agostino’s K^2 Test')
    stat, p = AgostinosK2Test(list(df[v].dropna()))
    
    
    print('------ LOG TRANSFORMATION')
    data = np.log(list(df[v].dropna()))
    if 0 in list(df[v].dropna()):
        data = np.log(list(df[v].dropna()+1))
    print('ShapiroTest')
    stat, p = ShapiroTest(data)
    print('D’Agostino’s K^2 Test')
    stat, p = AgostinosK2Test(data)
    

------------------------ CombPeck
ShapiroTest
Statistics=0.588, p=0.000
Sample does not look Gaussian (reject H0)
D’Agostino’s K^2 Test
Statistics=50.884, p=0.000
Sample does not look Gaussian (reject H0)
------ LOG TRANSFORMATION
ShapiroTest
Statistics=0.543, p=0.000
Sample does not look Gaussian (reject H0)
D’Agostino’s K^2 Test
Statistics=63.969, p=0.000
Sample does not look Gaussian (reject H0)
------------------------ CombColor
ShapiroTest
Statistics=0.155, p=0.000
Sample does not look Gaussian (reject H0)
D’Agostino’s K^2 Test
Statistics=121.290, p=0.000
Sample does not look Gaussian (reject H0)
------ LOG TRANSFORMATION
ShapiroTest
Statistics=0.155, p=0.000
Sample does not look Gaussian (reject H0)
D’Agostino’s K^2 Test
Statistics=121.290, p=0.000
Sample does not look Gaussian (reject H0)
------------------------ FeatherNeck
ShapiroTest
Statistics=0.790, p=0.000
Sample does not look Gaussian (reject H0)
D’Agostino’s K^2 Test
Statistics=101.545, p=0.000
Sample does not look Gauss

C:\Users\camil\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log
